In [58]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report

In [22]:
tonsil = pd.read_csv('/Volumes/Lukas_SSD/phenotyping_benchmark/datasets/BA_CODEX/quantification/tonsil_training/processed/B004_training_dryad_cleaned.csv')
be = pd.read_csv('/Volumes/Lukas_SSD/phenotyping_benchmark/datasets/BA_CODEX/quantification/BE_tonsil/processed/BE_Tonsil_l3_dryad_cleaned.csv')

In [8]:
be

,CHGA,PDL1,CD56,CK7,FoxP3,CD21,MUC1,PD1,CD11b,CD4,...,CD163,Bcatenin,CD138,Arginase1,CD73,CD206,MUC6,COX2,MMP9,cell_type
0,-0.015838,-0.262392,-0.328807,-0.030104,-0.169786,-0.195315,-0.144450,-0.314147,-0.306023,-0.681588,...,-0.189296,-0.395993,-0.229646,0.183429,-0.471492,0.368175,-0.137850,0.148542,-0.161118,Innate
1,-0.063117,-0.075990,1.852914,-0.030104,-0.169786,2.408519,-0.144450,1.413897,1.150271,-0.499367,...,-0.132587,5.211724,-0.229627,3.152803,-0.460310,0.406059,0.247144,0.549028,-0.114117,Innate
2,-0.070705,-0.262392,-0.362758,-0.030104,-0.169786,0.281066,-0.144449,-0.270863,-0.306020,-0.681588,...,-0.189296,-0.322642,-0.229654,-0.300006,0.798789,-0.238657,-0.137942,-0.167777,-0.161118,PDPN
3,-0.061208,-0.186271,-0.074156,-0.030104,-0.169786,-0.078391,-0.144447,-0.088802,0.294455,-0.681588,...,-0.188451,0.120267,-0.224748,1.068848,-0.256614,-0.230057,-0.130576,-0.069935,-0.161118,Endothelial
4,-0.070660,-0.123091,0.006811,-0.030104,0.100844,0.054867,-0.144450,-0.046886,2.337389,-0.588588,...,-0.189296,0.236230,-0.229654,3.108413,-0.451207,-0.165934,-0.126458,0.048613,-0.153662,Innate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219921,-0.070250,-0.262392,0.637652,-0.030104,-0.169786,-0.174148,-0.144450,0.044370,-0.306023,-0.572092,...,-0.189296,-0.420234,-0.209426,-0.263463,-0.367291,-0.110060,-0.137930,-0.096468,-0.161118,Innate
219922,-0.058163,-0.160154,0.202644,-0.030104,-0.169786,-0.219187,-0.144450,-0.201268,-0.306023,-0.619760,...,0.708596,-0.421789,-0.229654,-0.193195,0.916953,3.677105,-0.137942,-0.163933,-0.161118,Innate
219923,-0.067054,-0.214624,0.057478,-0.004263,-0.166512,-0.155890,0.100125,-0.122829,-0.301920,-0.255616,...,0.017257,-0.429658,-0.229654,-0.041300,-0.163893,-0.192927,-0.072235,-0.137067,-0.140883,Innate
219924,-0.013725,-0.138051,-0.057624,-0.030104,-0.169778,-0.309653,-0.121936,-0.334723,-0.306023,0.712992,...,-0.187530,0.092537,0.569675,-0.651138,-0.471492,-0.249438,-0.137942,-0.171432,-0.161118,Innate


In [23]:
tonsil.drop(columns=['x', 'y', 'tissue', 'donor', 'unique_region'], inplace=True)
be.drop(columns=['x', 'y', 'sample_name'], inplace=True)

In [9]:
be.columns

Index(['CHGA', 'PDL1', 'CD56', 'CK7', 'FoxP3', 'CD21', 'MUC1', 'PD1', 'CD11b',
       'CD4', 'CD31', 'CD25', 'CD15', 'CD20', 'Annexin A1', 'aSMA', 'CD11c',
       'Nestin', 'IDO', 'Cytokeratin', 'MUC5AC', 'Vimentin', 'CD36', 'HLADR',
       'BCL2', 'p63', 'CD3', 'CD45', 'CD8', 'CD57', 'aDefensin5', 'CD68',
       'CD34', 'CD38', 'Podoplanin', 'CD163', 'Bcatenin', 'CD138', 'Arginase1',
       'CD73', 'CD206', 'MUC6', 'COX2', 'MMP9', 'cell_type'],
      dtype='object')

In [10]:
tonsil.columns

Index(['MUC2', 'SOX9', 'MUC1', 'CD31', 'Synapto', 'CD49f', 'CD15', 'CHGA',
       'CDX2', 'ITLN1', 'CD4', 'CD127', 'Vimentin', 'HLADR', 'CD8', 'CD11c',
       'CD44', 'CD16', 'BCL2', 'CD3', 'CD123', 'CD38', 'CD90', 'aSMA', 'CD21',
       'NKG2D', 'CD66', 'CD57', 'CD206', 'CD68', 'CD34', 'aDef5', 'CD7',
       'CD36', 'CD138', 'CD45RO', 'Cytokeratin', 'CK7', 'CD117', 'CD19',
       'Podoplanin', 'CD45', 'CD56', 'CD69', 'Ki67', 'CD49a', 'CD163', 'CD161',
       'cell_type'],
      dtype='object')

In [12]:
matching_columns = tonsil.columns.intersection(be.columns)
print(matching_columns)

Index(['MUC1', 'CD31', 'CD15', 'CHGA', 'CD4', 'Vimentin', 'HLADR', 'CD8',
       'CD11c', 'BCL2', 'CD3', 'CD38', 'aSMA', 'CD21', 'CD57', 'CD206', 'CD68',
       'CD34', 'CD36', 'CD138', 'Cytokeratin', 'CK7', 'Podoplanin', 'CD45',
       'CD56', 'CD163', 'cell_type'],
      dtype='object')


In [13]:
tonsil

,MUC2,SOX9,MUC1,CD31,Synapto,CD49f,CD15,CHGA,CDX2,ITLN1,...,CD19,Podoplanin,CD45,CD56,CD69,Ki67,CD49a,CD163,CD161,cell_type
0,-0.241565,0.538240,-0.145700,-0.254499,-0.312923,-0.440809,-0.083480,-0.073498,-0.182247,-0.395115,...,-0.094368,-0.208229,-0.378092,-0.248421,-0.255296,-0.270736,-0.574298,-0.274995,-0.083926,Enterocyte
1,-0.231567,1.053268,0.015944,-0.248268,-0.314157,0.461514,-0.094145,-0.087275,0.246550,-0.292916,...,-0.090799,-0.214242,-0.457434,-0.242000,-0.246320,0.161482,-0.484751,-0.265804,-0.052165,Enterocyte
2,0.207531,0.526999,-0.057481,-0.249034,-0.325943,0.033951,-0.094142,-0.086326,-0.005325,-0.288484,...,-0.095140,-0.216210,-0.171199,-0.247935,-0.250225,0.360991,-0.517455,-0.274345,-0.062414,Enterocyte
3,-0.076672,-0.420316,-0.051284,-0.252876,-0.302395,1.100307,-0.093069,-0.081959,0.005021,-0.355146,...,-0.096980,-0.218523,-0.343095,-0.243046,-0.129018,0.137150,-0.460114,-0.274582,-0.063367,Enterocyte
4,-0.235334,0.376148,0.025601,-0.251348,-0.316774,-0.074358,-0.094141,-0.080630,-0.029893,-0.342737,...,-0.092804,-0.210291,-0.330390,-0.248646,-0.247002,-0.041417,-0.519680,-0.273687,-0.047192,Enterocyte
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248280,-0.301417,-0.480617,-0.485732,-0.247548,-0.075596,-0.601329,-0.296372,-0.121456,-0.298825,-0.407754,...,0.454352,-0.302954,0.108039,-0.183081,-0.127508,-0.275793,-0.448915,-0.329448,-0.110390,Plasma
248281,-0.302128,-0.469591,-0.486481,-0.201661,-0.252108,-0.632735,-0.291958,-0.089449,-0.345515,-0.399732,...,-0.161266,-0.315273,-0.144110,-0.280050,-0.275998,-0.298807,-0.648830,-0.263899,-0.104248,Plasma
248282,-0.300624,-0.505039,-0.481408,-0.206155,-0.257355,-0.611439,-0.297305,-0.096474,-0.338581,-0.430508,...,-0.075536,-0.337855,-0.202179,-0.278031,-0.298619,-0.290010,-0.616562,-0.280333,-0.082285,Plasma
248283,-0.241006,-0.217832,-0.350860,-0.235930,0.407688,-0.585294,-0.297135,-0.069509,-0.099756,-0.256268,...,-0.104034,0.172151,0.070700,-0.194168,-0.287577,0.012376,-0.387270,-0.120730,-0.058423,Plasma


In [26]:
tonsil['cell_type'].unique()

array(['Enterocyte', 'Goblet', 'Neuroendocrine', 'SmoothMuscle',
       'Lymphatic', 'T', 'Macrophage', 'DC', 'Neutrophil', 'Endothelial',
       'TA', 'Plasma', 'Stroma', 'Nerve', 'ICC', 'Paneth', 'CD7_Immune',
       'Enterocyte_ITLN1p', 'Enterocyte_CD57p', 'B'], dtype=object)

In [25]:
tonsil['cell_type'] = ['T' if x == 'CD8T' or x == 'CD4T' else x for x in tonsil['cell_type']]


In [27]:
be['cell_type'].unique()

array(['Innate', 'PDPN', 'Endothelial', 'B', 'T', 'Squamous_epithelial',
       'Stroma', 'SmoothMuscle', 'Plasma', 'Nerve', 'Glandular_epi',
       'Secretory_epithelial', 'Paneth'], dtype=object)

In [28]:
matching_celltypes = [x for x in tonsil['cell_type'].unique() if x in be['cell_type'].unique()]

In [29]:
matching_celltypes

['SmoothMuscle',
 'T',
 'Endothelial',
 'Plasma',
 'Stroma',
 'Nerve',
 'Paneth',
 'B']

In [30]:
tonsil_filtered = tonsil[tonsil['cell_type'].isin(matching_celltypes)]
be_filtered = be[be['cell_type'].isin(matching_celltypes)]

In [34]:
tonsil_filtered = tonsil_filtered[matching_columns]
be_filtered = be_filtered[matching_columns]

In [35]:
tonsil_filtered

,MUC1,CD31,CD15,CHGA,CD4,Vimentin,HLADR,CD8,CD11c,BCL2,...,CD34,CD36,CD138,Cytokeratin,CK7,Podoplanin,CD45,CD56,CD163,cell_type
37881,-0.228511,-0.252109,0.044017,-0.087527,0.304383,0.665620,0.504706,-0.280019,-0.171794,-0.426575,...,-0.279160,-0.198039,-0.237497,-0.524284,-0.110343,-0.017378,-0.370005,-0.225287,1.491814,SmoothMuscle
37882,-0.293809,-0.238418,-0.094144,-0.087527,-0.335094,-0.323415,0.873042,-0.281641,-0.218573,-0.448709,...,-0.266312,0.440505,-0.246115,-0.529549,-0.133266,-0.102148,-0.457434,-0.260906,0.378695,SmoothMuscle
37883,-0.339154,-0.063026,-0.094142,-0.085339,-0.273327,2.321928,0.693147,-0.272828,-0.141934,-0.448707,...,-0.092901,-0.006511,-0.235600,-0.522737,-0.099458,0.713742,-0.303640,-0.226309,-0.286061,SmoothMuscle
37884,-0.290357,-0.244592,-0.094145,-0.062334,-0.335093,1.965262,-0.168415,-0.263199,-0.117196,-0.448705,...,-0.225634,-0.156241,-0.237528,-0.512758,-0.092654,0.132233,-0.457433,-0.237552,-0.286061,SmoothMuscle
37885,-0.308205,-0.256213,-0.028256,-0.087530,-0.335093,0.244522,-0.371739,-0.281641,-0.218571,-0.426882,...,-0.276248,-0.224149,-0.243494,-0.526917,-0.119919,-0.048857,-0.457434,-0.260907,-0.286061,SmoothMuscle
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248279,-0.441320,-0.224256,-0.298145,-0.087289,-0.368257,0.566626,-0.287231,-0.184904,-0.259679,0.882401,...,-0.270469,-0.206862,0.104786,-0.512507,-0.132296,-0.145276,-0.306065,0.197052,-0.281431,Plasma
248280,-0.485732,-0.247548,-0.296372,-0.121456,-0.446972,0.068351,-0.473117,-0.176043,-0.310694,-0.074675,...,-0.296392,-0.215866,0.306360,-0.498897,-0.132417,-0.302954,0.108039,-0.183081,-0.329448,Plasma
248281,-0.486481,-0.201661,-0.291958,-0.089449,-0.273751,0.538100,-0.246118,-0.186692,-0.234161,-0.216009,...,-0.294662,-0.172433,0.227266,-0.508494,-0.128657,-0.315273,-0.144110,-0.280050,-0.263899,Plasma
248282,-0.481408,-0.206155,-0.297305,-0.096474,-0.367354,-0.010073,-0.306902,-0.186519,-0.255714,-0.303270,...,-0.280874,-0.138368,-0.070400,-0.497548,-0.154558,-0.337855,-0.202179,-0.278031,-0.280333,Plasma


In [36]:
be_filtered

,MUC1,CD31,CD15,CHGA,CD4,Vimentin,HLADR,CD8,CD11c,BCL2,...,CD34,CD36,CD138,Cytokeratin,CK7,Podoplanin,CD45,CD56,CD163,cell_type
3,-0.144447,-0.080328,0.192008,-0.061208,-0.681588,-0.370965,-0.507189,-0.169657,-0.310194,-0.680829,...,-0.099757,0.576276,-0.224748,-0.12588,-0.030104,0.098176,-0.681172,-0.074156,-0.188451,Endothelial
10,-0.144381,-0.117302,-0.169447,-0.069567,-0.617458,-0.370965,-0.490953,-0.243808,-0.307226,-0.609818,...,-0.099758,-0.052865,-0.229646,-0.12588,-0.030104,-0.392957,-0.708878,-0.371648,-0.180858,B
12,-0.144436,-0.113342,-0.250582,-0.058668,-0.681588,-0.206051,0.911798,-0.375146,-0.209907,-0.680829,...,-0.092330,-0.119986,-0.227810,-0.12588,-0.030104,-0.403654,-0.704541,-0.453269,-0.167237,B
15,-0.144447,-0.117152,-0.250582,-0.035663,-0.681588,-0.186212,-0.312253,-0.307981,-0.345933,-0.641537,...,-0.088935,0.022512,-0.199877,-0.12588,-0.030104,-0.435641,-0.414270,-0.418899,-0.159227,B
17,-0.144438,-0.116914,-0.250582,-0.055208,-0.093120,-0.213406,-0.514585,-0.375146,-0.312235,-0.680829,...,-0.099791,-0.335711,-0.229654,-0.12588,-0.030104,-0.381067,-0.716825,-0.477303,0.269767,T
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219553,-0.144450,-0.117302,-0.250582,-0.049541,-0.681342,-0.370965,-0.522354,-0.352749,-0.345933,-0.625015,...,-0.099791,-0.322330,-0.229654,-0.12588,-0.030104,-0.216406,-0.716825,-0.230642,-0.189296,SmoothMuscle
219554,-0.144450,-0.117302,-0.250582,-0.061202,-0.643772,-0.370573,-0.522354,-0.337325,-0.345933,-0.481417,...,-0.099791,-0.259909,-0.229654,-0.12588,-0.030104,-0.418531,-0.716825,-0.244742,-0.189296,SmoothMuscle
219555,-0.144450,-0.117302,-0.250582,-0.063325,-0.634725,-0.370965,-0.522354,-0.362500,-0.345933,-0.342847,...,-0.099791,-0.015006,-0.229654,-0.12588,-0.030104,-0.139654,-0.716825,1.501796,-0.189296,Nerve
219556,-0.144450,-0.117302,-0.250582,-0.059603,-0.642330,3.568528,-0.522354,-0.306011,-0.345933,-0.499075,...,-0.099791,1.754181,-0.229654,-0.12588,-0.030104,-0.355462,-0.716825,1.023348,-0.189296,Stroma


In [37]:
scaler = StandardScaler()

In [41]:
tonsil_filtered['cell_type'] = tonsil_filtered['cell_type'].astype('category')
be_filtered['cell_type'] = be_filtered['cell_type'].astype('category')

/var/folders/ry/jl4ll0zx1l709rh4r7_0cq6r0000gn/T/ipykernel_17834/3052414476.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tonsil_filtered['cell_type'] = tonsil_filtered['cell_type'].astype('category')


In [48]:
X_train = tonsil_filtered.drop(columns=['cell_type'])
X_test = be_filtered.drop(columns=['cell_type'])
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
y_train = tonsil_filtered['cell_type']
y_test = be_filtered['cell_type']

In [49]:
label_encoder = LabelEncoder()

# Fit and transform the training labels
y_train_encoded = label_encoder.fit_transform(y_train)

# Transform the test labels
y_test_encoded = label_encoder.transform(y_test)


In [61]:
xgb = XGBClassifier( n_thread = 8, max_depth=10, tree_method='auto', scale_pos_weight=1)

In [62]:
xgb.fit(X_train_scaled, y_train_encoded)

/Users/lukashat/miniforge3/envs/xgboost/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [14:08:04] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "n_thread", "scale_pos_weight" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None, n_thread=8,
              num_parallel_tree=None, ...)

In [63]:
ypred = xgb.predict(X_test_scaled)

In [64]:
accuracy = accuracy_score(y_test_encoded, ypred)

In [65]:
accuracy

0.6101386489035802

In [66]:
y_test_decoded = label_encoder.inverse_transform(y_test_encoded)
ypred_decoded = label_encoder.inverse_transform(ypred)

# Generate the classification report
report = classification_report(y_test_decoded, ypred_decoded)
print(report)

              precision    recall  f1-score   support

           B       0.91      0.12      0.21     31867
 Endothelial       0.54      0.74      0.62      9476
       Nerve       0.44      0.71      0.54      2328
      Paneth       0.00      0.00      0.00       275
      Plasma       0.20      0.90      0.32      3420
SmoothMuscle       0.50      0.92      0.65      9175
      Stroma       0.57      0.44      0.50     13164
           T       0.77      0.86      0.82     50094

    accuracy                           0.61    119799
   macro avg       0.49      0.59      0.46    119799
weighted avg       0.72      0.61      0.57    119799

